In [3]:
import mindspore as ms
from mindspore.mindrecord import FileWriter
# 载入mindspore的默认数据集
import mindspore.dataset as ds

# 图像转化用算子
####____####
from PIL import Image
import mindspore.dataset.vision.c_transforms as cvision
import mindspore.dataset.vision as vision
import mindspore.dataset.transforms.c_transforms as ctrans

from mindspore.common import dtype as mstype
# mindspore的tensor
from mindspore import Tensor

# 各类网络层都在nn里面
import mindspore.nn as nn
# 参数初始化的方式

from mindspore.common.initializer import TruncatedNormal
# 设置mindspore运行的环境
from mindspore import context
# 引入训练时候会使用到回调函数，如checkpoint, lossMoniter
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor, Callback
# 引入模型
from mindspore.train import Model
# 引入评估模型的包
from mindspore.nn.metrics import Accuracy

# numpy
import numpy as np
# 画图用
import matplotlib.pyplot as plt
import os
from io import BytesIO
import glob
import cv2

import requests 
import pandas as pd
from tqdm import tqdm

from mindspore.common import dtype as mstype
from mindspore import context

data_dict = {'complex': [0,0,0,1,0,0],
 'frog_eye_leaf_spot': [0,0,1,0,0,0],
 'frog_eye_leaf_spot complex':[0,0,1,1,0,0],
 'healthy':[1,0,0,0,0,0],
 'powdery_mildew':[0,0,0,0,1,0],
 'powdery_mildew complex':[0,0,0,1,1,0],
 'rust':[0,0,0,0,0,1],
 'rust complex':[0,0,0,1,0,1],
 'rust frog_eye_leaf_spot':[0,0,1,0,0,1],
 'scab':[0,1,0,0,0,0],
 'scab frog_eye_leaf_spot':[0,1,1,0,0,0],
 'scab frog_eye_leaf_spot complex':[0,1,1,1,0,0]}

device_target = ms.context.get_context('device_target')
# ms.set_context(mode=ms.GRAPH_MODE, device_target='GPU')
# 确定图模型是否下沉到芯片上
dataset_sink_mode = True #if device_target in ['Ascend','GPU'] else False
# 设置模型的设备与图的模式
ms.set_context(mode=context.GRAPH_MODE, device_target=device_target)

In [2]:
def read_data(data_type, data_dict, small_batch=None):
    
    data_dir = 'plant_dataset/'
    csv_fname = os.path.join(data_dir, data_type ,data_type + '_label.csv')
    csv_data = pd.read_csv(csv_fname)
    csv_data = csv_data.set_index('images')
    
    images, labels = [], []
    if small_batch is None:
        num = len(csv_data)
    else:
        num = small_batch
        
    for i in tqdm(range(num)):
        img_name = csv_data.index[i]
        target = csv_data.labels[i]
        image_dir = os.path.join(data_dir, data_type , 'images', img_name)
        label = np.array(data_dict[target])
        images.append(image_dir)
        labels.append(label)
        
    return images, labels

In [3]:
from mindspore.dataset.transforms import c_transforms
import mindspore.dataset.vision.c_transforms as c_vision
import mindspore.dataset.vision.py_transforms as py_vision
transforms_list = [c_vision.RandomHorizontalFlip(0.5),
                   py_vision.RandomErasing()]
composed_transform = c_transforms.Compose([c_transforms.RandomApply(transforms_list, prob=0.6),ms.dataset.vision.c_transforms.HWC2CHW()])
composed_transform_val = [ms.dataset.vision.c_transforms.HWC2CHW()]

In [4]:
class Plant_Dataset():
    def __init__(self, data_type, small_batch = None):
        self.index = 0
        self.small_batch = small_batch
        self.img, self.label = read_data(data_type, data_dict, self.small_batch)
        self.images = {}
        for i in range(0,len(self.img)):
            self.images[i] = cv2.imread(self.img[i])
            self.images[i] = cv2.resize(self.images[i],(224,224))
            self.images[i] = self.images[i][:,:,[2,1,0]]
        
        
        # self._img_all = cv2.imread(self._data)
    def __next__(self):
        if self.index >= len(self.img):
            raise StopIteration
        else:           
            item = (ms.Tensor(self.images[self.index],ms.float32), ms.Tensor(self.label[self.index],ms.float32))
            self.index += 1
            return item

    def __iter__(self):
        self.index = 0
        return self

    def __len__(self):
        return len(self.img)

In [5]:
train_plant_set = Plant_Dataset('train', None)
val_plant_set = Plant_Dataset('val', None)
train_dataset = ds.GeneratorDataset(source=train_plant_set, column_names=["image", "label"], num_parallel_workers=6)# channel, height, width
val_dataset = ds.GeneratorDataset(source=val_plant_set, column_names=["image", "label"], num_parallel_workers=6)


100%|██████████| 600/600 [00:00<00:00, 34292.87it/s]


In [6]:
train_set = train_dataset.map(operations=composed_transform, input_columns=["image"], num_parallel_workers=6)
val_set = val_dataset.map(operations=composed_transform_val, input_columns=["image"], num_parallel_workers=6)

batch_size = 32
epoch = 100

train_set = train_set.batch(batch_size, drop_remainder=True, num_parallel_workers=6)
val_set = val_set.batch(batch_size, drop_remainder=True, num_parallel_workers=6)

In [5]:
def _weight_variable(shape, factor=0.01):
    init_value = np.random.randn(*shape).astype(np.float32) * factor
    return ms.Tensor(init_value)


def _conv3x3(in_channel, out_channel, stride=1):
    weight_shape = (out_channel, in_channel, 3, 3)
    weight = _weight_variable(weight_shape)
    return nn.Conv2d(in_channel, out_channel,
                     kernel_size=3, stride=stride, padding=0, pad_mode='same', weight_init=weight)


def _conv1x1(in_channel, out_channel, stride=1):
    weight_shape = (out_channel, in_channel, 1, 1)
    weight = _weight_variable(weight_shape)
    return nn.Conv2d(in_channel, out_channel,
                     kernel_size=1, stride=stride, padding=0, pad_mode='same', weight_init=weight)


def _conv7x7(in_channel, out_channel, stride=1):
    weight_shape = (out_channel, in_channel, 7, 7)
    weight = _weight_variable(weight_shape)
    return nn.Conv2d(in_channel, out_channel,
                     kernel_size=7, stride=stride, padding=0, pad_mode='same', weight_init=weight)


def _bn(channel):
    return nn.BatchNorm2d(channel, eps=1e-4, momentum=0.9,
                          gamma_init=1, beta_init=0, moving_mean_init=0, moving_var_init=1)


def _bn_last(channel):
    return nn.BatchNorm2d(channel, eps=1e-4, momentum=0.9,
                          gamma_init=0, beta_init=0, moving_mean_init=0, moving_var_init=1)


def _fc(in_channel, out_channel):
    weight_shape = (out_channel, in_channel)
    weight = _weight_variable(weight_shape)
    return nn.Dense(in_channel, out_channel, has_bias=True, weight_init=weight, bias_init=0)


class ResidualBlock(nn.Cell):
    """
    ResNet V1 residual block definition.
    Args:
        in_channel (int): Input channel.
        out_channel (int): Output channel.
        stride (int): Stride size for the first convolutional layer. Default: 1.
    Returns:
        Tensor, output tensor.
    Examples:
        >>> ResidualBlock(3, 256, stride=2)
    """
    expansion = 4

    def __init__(self,
                 in_channel,
                 out_channel,
                 stride=1):
        super(ResidualBlock, self).__init__()

        channel = out_channel // self.expansion
        self.conv1 = _conv1x1(in_channel, channel, stride=1)
        self.bn1 = _bn(channel)

        self.conv2 = _conv3x3(channel, channel, stride=stride)
        self.bn2 = _bn(channel)

        self.conv3 = _conv1x1(channel, out_channel, stride=1)
        self.bn3 = _bn_last(out_channel)

        self.relu = nn.ReLU()

        self.down_sample = False

        if stride != 1 or in_channel != out_channel:
            self.down_sample = True
        self.down_sample_layer = None

        if self.down_sample:
            self.down_sample_layer = nn.SequentialCell([_conv1x1(in_channel, out_channel, stride),
                                                        _bn(out_channel)])
        self.add = ops.Add()

    def construct(self, x):  # pylint: disable=missing-docstring
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.down_sample:
            identity = self.down_sample_layer(identity)

        out = self.add(out, identity)
        out = self.relu(out)

        return out


class ResNet(nn.Cell):
   
    def __init__(self,
                 block,
                 layer_nums,
                 in_channels,
                 out_channels,
                 strides,
                 num_classes):
        super(ResNet, self).__init__()

        if not len(layer_nums) == len(in_channels) == len(out_channels) == 4:
            raise ValueError("the length of layer_num, in_channels, out_channels list must be 4!")

        self.conv1 = _conv7x7(3, 64, stride=2)
        self.bn1 = _bn(64)
        self.relu = ops.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, pad_mode="same")

        self.layer1 = self._make_layer(block,
                                       layer_nums[0],
                                       in_channel=in_channels[0],
                                       out_channel=out_channels[0],
                                       stride=strides[0])
        self.layer2 = self._make_layer(block,
                                       layer_nums[1],
                                       in_channel=in_channels[1],
                                       out_channel=out_channels[1],
                                       stride=strides[1])
        self.layer3 = self._make_layer(block,
                                       layer_nums[2],
                                       in_channel=in_channels[2],
                                       out_channel=out_channels[2],
                                       stride=strides[2])
        self.layer4 = self._make_layer(block,
                                       layer_nums[3],
                                       in_channel=in_channels[3],
                                       out_channel=out_channels[3],
                                       stride=strides[3])

        self.mean = ops.ReduceMean(keep_dims=True)
        self.flatten = nn.Flatten()
        self.end_point = _fc(out_channels[3], num_classes)
        self.sigmoid = ops.Sigmoid()

    def _make_layer(self, block, layer_num, in_channel, out_channel, stride):
      
        layers = []

        resnet_block = block(in_channel, out_channel, stride=stride)
        layers.append(resnet_block)

        for _ in range(1, layer_num):
            resnet_block = block(out_channel, out_channel, stride=1)
            layers.append(resnet_block)

        return nn.SequentialCell(layers)

    def construct(self, x):  # pylint: disable=missing-docstring
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        c1 = self.maxpool(x)

        c2 = self.layer1(c1)
        c3 = self.layer2(c2)
        c4 = self.layer3(c3)
        c5 = self.layer4(c4)

        out = self.mean(c5, (2, 3))
        out = self.flatten(out)
        out = self.end_point(out)
        out = self.sigmoid(out)
        return out

    

def resnet18(class_num=6):
    
    return ResNet(ResidualBlock,
                  [2, 2, 2, 2],
                  [64, 64, 128, 256],
                  [64, 128, 256, 512],
                  [1, 2, 2, 2],
                  class_num)

def resnet50(class_num=6):
    
    return ResNet(ResidualBlock,
                  [3, 4, 6, 3],
                  [64, 256, 512, 1024],
                  [256, 512, 1024, 2048],
                  [1, 2, 2, 2],
                  class_num)


def resnet101(class_num=6):
    
    return ResNet(ResidualBlock,
                  [3, 4, 23, 3],
                  [64, 256, 512, 1024],
                  [256, 512, 1024, 2048],
                  [1, 2, 2, 2],
                  class_num)

In [8]:
import mindspore.ops as ops
network = resnet18(6)
net_loss = nn.BCELoss(reduction="mean")
# 优化器为Adam
net_opt = nn.Adam(params=network.trainable_params(), learning_rate=0.001) 
from sklearn.metrics import accuracy_score
from utils import *
class MultilabelAccuracy2(nn.Metric):
    def __init__(self):
        super(MultilabelAccuracy2, self).__init__()
        self.clear()

    def clear(self):
        """初始化变量_abs_error_sum和_samples_num"""
        self.correct_sum = 0  # 保存预测正确的样本数
        self.samples_num = 0  # 累计数据量

    @nn.rearrange_inputs
    def update(self, *inputs):
        """更新_abs_error_sum和_samples_num"""
        y_pred = inputs[0].asnumpy()
        y = inputs[1].asnumpy()
        y_pred = np.where(y_pred > 0.5, 1, 0)
        count = accuracy_score(y,y_pred)*y.shape[0]
        self.correct_sum += count

        # 样本的总数
        self.samples_num += y.shape[0]
    def eval(self):
        """计算最终评估结果"""
        return self.correct_sum / self.samples_num

In [9]:
from utils import *
class EvalCallBack(Callback):
    def __init__(self, model, eval_dataset, eval_per_epoch, epoch_per_eval):
        self.model = model
        self.eval_dataset = eval_dataset
        self.eval_per_epoch = eval_per_epoch
        self.epoch_per_eval = epoch_per_eval

    def epoch_end(self, run_context):
        cb_param = run_context.original_args()
        cur_epoch = cb_param.cur_epoch_num
        if cur_epoch % self.eval_per_epoch == 0:
            acc = self.model.eval(self.eval_dataset, dataset_sink_mode=False)
            self.epoch_per_eval["epoch"].append(cur_epoch)
            self.epoch_per_eval["acc"].append(acc["Accuracy"])
            print(acc)


# # 设置CheckpointConfig，callback函数。save_checkpoint_steps=训练总数/batch_size
config_ck = CheckpointConfig(save_checkpoint_steps=93,
                             keep_checkpoint_max=10)
ckpoint_cb = ModelCheckpoint(prefix="checkpoint_resnet18_original", directory='./results',config=config_ck)
# 建立可训练模型
model = Model(network=network,loss_fn=net_loss,optimizer=net_opt, metrics={"Accuracy": MultilabelAccuracy2()})
eval_per_epoch = 1
epoch_per_eval = {"epoch": [], "acc": []}
eval_cb = EvalCallBack(model, val_set, eval_per_epoch, epoch_per_eval)
print("============== Starting Training ==============")
model.train(2000, train_set,callbacks=[ckpoint_cb, LossMonitor(per_print_times=1),eval_cb],dataset_sink_mode=dataset_sink_mode)

============== Starting Training ==============


[WARNING] MD(1707805,fffe657fa1e0,python):2022-11-06-12:05:10.581.349 [mindspore/ccsrc/minddata/dataset/engine/datasetops/device_queue_op.cc:776] DetectPerBatchTime] Bad performance attention, it takes more than 25 seconds to fetch a batch of data from dataset pipeline, which might result `GetNext` timeout problem. You may test dataset processing performance(with creating dataset iterator) and optimize it.


epoch: 1 step: 93, loss is 0.3918694853782654
{'Accuracy': 0.08333333333333333}
epoch: 2 step: 93, loss is 0.3713840842247009
{'Accuracy': 0.11805555555555555}
epoch: 3 step: 93, loss is 0.3769158720970154
{'Accuracy': 0.1371527777777778}
epoch: 4 step: 93, loss is 0.3700260519981384
{'Accuracy': 0.1597222222222222}
epoch: 5 step: 93, loss is 0.3664950430393219
{'Accuracy': 0.1527777777777778}
epoch: 6 step: 93, loss is 0.35947322845458984
{'Accuracy': 0.1684027777777778}
epoch: 7 step: 93, loss is 0.35554227232933044
{'Accuracy': 0.1701388888888889}
epoch: 8 step: 93, loss is 0.3528251349925995
{'Accuracy': 0.24131944444444445}
epoch: 9 step: 93, loss is 0.34397047758102417
{'Accuracy': 0.265625}
epoch: 10 step: 93, loss is 0.32916688919067383
{'Accuracy': 0.28125}
epoch: 11 step: 93, loss is 0.3245631158351898
{'Accuracy': 0.265625}
epoch: 12 step: 93, loss is 0.3322222828865051
{'Accuracy': 0.2673611111111111}
epoch: 13 step: 93, loss is 0.2918356955051422
{'Accuracy': 0.33159722222

In [7]:
##### 将模型参数存入parameter的字典中，这里加载的是上面训练过程中保存的模型参数
import mindspore.ops as ops
param_dict = ms.load_checkpoint("./results/checkpoint_resnet18_original_12-1996_93.ckpt")

net1 = resnet18(6)

ms.load_param_into_net(net1, param_dict)

net1_opt=nn.Adam(params=network.trainable_params(), learning_rate=0.001)
net1_loss = nn.BCELoss(reduction="mean")
model1=ms.Model(net1,net1_loss,net1_opt,metrics={"Accuracy": MultilabelAccuracy2()})
print("============== 开始测试 ==============")
result=model1.eval(val_set,dataset_sink_mode=dataset_sink_mode)
print(result)

NameError: name 'network' is not defined